# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(stride=2,kernel_size=2))
        self.dense = nn.Sequential(nn.Linear(14*14*128,1024), nn.ReLU(),nn.Dropout(p = 0.5),nn.Linear(1024,10))
        
    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1, 14*14*128)
        x = self.dense(x)
        return x


model = SimpleNet()
model = model.cuda()


# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    running_correct = 0
    print("Epoch {}/{}".format(epoch+1,NUM_EPOCHS))
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images, labels = torch.autograd.Variable(images.cuda()),torch.autograd.Variable(labels.cuda()) 
        outputs = model(images)
        _,pred = torch.max(outputs.data,1)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        running_correct += torch.sum(pred == labels.data)
    
    testing_correct = 0
    for data in tqdm(test_loader):
        x_test,y_test = data
        x_test,y_test = torch.autograd.Variable(x_test.cuda()),torch.autograd.Variable(y_test.cuda())
 
        outputs = model(x_test)
        _,pred = torch.max(outputs.data,1)
        testing_correct += torch.sum(pred == y_test.data)
 
    print("Loss is :{:.4f},Train Accuracy is : {:.4f}%, Test Accuracy is {:.4f}".format
          (running_loss/len(train_dataset),100*running_correct/len(train_dataset),100*testing_correct/len(test_dataset)))


  0%|▎                                                                                 | 2/468 [00:00<00:29, 15.92it/s]

Epoch 1/10


  1%|▌                                                                                 | 3/468 [00:00<00:24, 18.62it/s]

Loss is :0.0001,Train Accuracy is : 99.6383%, Test Accuracy is 98.7700
Epoch 2/10


  1%|▌                                                                                 | 3/468 [00:00<00:23, 19.92it/s]

...

Loss is :0.0000,Train Accuracy is : 99.6850%, Test Accuracy is 98.7200
Epoch 9/10


  1%|▌                                                                                 | 3/468 [00:00<00:24, 18.68it/s]

Loss is :0.0000,Train Accuracy is : 99.7033%, Test Accuracy is 98.8400
Epoch 10/10


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 45.23it/s]

Loss is :0.0000,Train Accuracy is : 99.7000%, Test Accuracy is 98.4200


#### Q5:
Please print the training and testing accuracy.